Чат-бот в телеграме ищет по странице в фейсбуке все посты про сдачу квартиры

In [81]:
import telebot
import pandas as pd
import facebook

In [82]:
access_token = 'EAACEdEose0cBAJah4e3gJOdWNeoJ7jbNWTX4BwpIpvPQBR9upnyGdKBsa5gnDhTiQgZBb881GkYIxVnSc4sKz9iImzQTHDot98lCPZB8MwZA9V3Gj3ZByr2G31aFBhZCoWIT8KgzthjasBWRZADsZBphRZC7mDSEGHxikvFAA2xyWywoqxab6gyrPrL6udQs0lsZD'

In [60]:
posts[1]['message'] #основное сообщение поста

'Хэййй пипл!!) \nИщем соседку или соседа или пару для подселения в соседнюю комнату!с 24го июля!без посредников и агентов! 5 минут пешком до метро " Водный стадион" Комнатка маленькая с диваном, закрывается! Что хотелось бы: чтобы соседка/сосед был/а опрятная, аккуратная и честная, во время платила квартплату.только славяне.\nАренда 15тыс. Залог. 10тыс. \nМы- молодая пара, иногда курим кальян, в основном уезжаем на работу и дома спим. Любим готовить и поддерживать чистоту в квартире.:)\nФотографии квартиры и комнаты отправлю в личку!\nАГЕНТАМ НЕ БЕСПОКОИТЬ!!\nПусть наш дом станет и вашим домом:)\nЗа репост +200 к карме:)\nВсем мир!)'

In [83]:
def fb_pasrsing(access_token,chat_id):
    graph = facebook.GraphAPI(access_token) #через api нет ссылок на посты, только id

    # id of 'Flats for friends' page
    page = '509679185734909'

    profile = graph.get_object(page)
    posts = graph.get_connections(profile['id'], 'feed')['data'] #feed - новостная лента
    
    try:
        seen_posts = list(pd.read_csv('seen_posts'+str(chat_id)+'.csv').post_id)
    except:
        seen_posts = []
        file = open('seen_posts'+str(chat_id)+'.csv', 'w')
        file.write('post_id')
        file.close()
    seen_posts_new, posts_to_show = [], []

    for post in posts:
        if len(set(seen_posts)&{post['id']})==0:
            seen_posts_new.append(post['id'])
            try:
                bag_of_words = set(posts[0]['message'].lower().split(' '))

                if (len(bag_of_words&{'сдам','сдаю','сдается'})>0)\
                &  (len(bag_of_words&{'квартира','квартиру'})>0):
                    link = 'https://www.facebook.com/groups/%s/permalink/%s/' % tuple(post['id'].split('_')) #вставляет два числа из id поста вместо %s и получается ссылка
                    posts_to_show.append(link)
            except KeyError:
                pass
    pd.DataFrame(seen_posts+seen_posts_new,columns=['post_id']).to_csv('seen_posts'+str(chat_id)+'.csv', index=False) 
    return(posts_to_show)

### Сбор ссылок телеграм-ботом.

В телеграме пишем /newbot в чат botfather. Придумываем имя чат-боту и получаем токену.

In [87]:
tg_token = '444864632:AAEbK-YXHH92FWE7Qrks9j9CfhC-VRYRlTU'

In [ ]:
bot = telebot.TeleBot(tg_token)

@bot.message_handler(commands=['start'])

def handle_start(message):
   bot.send_message(message.chat.id, "Hi!")
    
@bot.message_handler(commands=['search'])

def search_flats(message):
    posts_to_show = fb_pasrsing(access_token, message.chat.id)
    if len(posts_to_show)==0:
        bot.send_message(message.chat.id, 'Since the last request, there were no new ads, sorry :(')
    else:
        for link in posts_to_show:
            bot.send_message(message.chat.id, link)
            
if __name__ == '__main__':
   bot.polling(none_stop=True)